In [1]:
import pandas as pd
import os
import psycopg2
from psycopg2.extras import DictCursor
import configparser
import logging

//anaconda/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
class PGconnect:
    def __init__(self, db, port=5432):
        self.db = db
        self.env = os.environ
        self.port = port
        logging.basicConfig()
        self.log = logging.getLogger(__name__)
        self.log.debug("PGconnect Initialized!")

        try:
            self.con = self.connect()
            self.log.debug("Connected to {}!".format(db))
        except:
            self.log.error("Could not make a connection to {}!".format(db))

    def connect(self, db='cape-dev'):
        """
        Connect to database.
        Returns
        -------
        con: connection
            psycopg2 `connection` object
        """
        cp = configparser.ConfigParser()
        try:
            cp.read('/Users/{user}/config.ini'.format(user=self.env['USER']))
        except:
            self.log.error("could not read config file for {user}".format(env['USER']))

        connections = {}

        for s in cp.sections():
            connections[s] = {i[0]: i[1] for i in cp.items(s)}

        conn = psycopg2.connect(**dict(cp.items(db)))

        return conn
    
    def get_df(self, query):
        """
        Get a dataframe from query.
        Parameters
        ----------
        query: str
            SQL query statement
        Returns
        -------
        df: pandas.core.frame.DataFrame
            Pandas dataframe object
        """
        self.log.debug('Requesting a dataframe for query: {}'.format(query))
        return pd.read_sql_query(query, con=self.con)